In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import os

from matplotlib.lines import Line2D
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib.colors as mcolors
import math, bisect
import matplotlib.colors as mcolors

from pandas.plotting import parallel_coordinates
from pandas.api.types import CategoricalDtype
from itertools import product
import warnings
warnings.filterwarnings("ignore")

# spatial analysis
import geopandas as gpd

In [2]:
# Define Constants
resid = ['resid heating','resid cooling','resid others']
food = ['FoodDemand_NonStaples','FoodDemand_Staples']

fut_years = [2020,2025,2030,2035,2040,2045,2050,2055,2060,2065,2070,2075,2080,2085,2090,2095,2100]

# Constants for Unit Conversion
EJ_to_GJ = 10**9
GJ_to_EJ = 1/EJ_to_GJ
EJ_to_KWh = 277777777777.78

Mt_to_kg = 10**9
kg_to_Mt = 1/Mt_to_kg

Kcal_to_Mcal = 10**3
Pcal_to_Mcal = 10**9

USD1975_to_USD2020 = 4.8106 # https://www.in2013dollars.com/us/inflation/1975?endYear=2020&amount=100
USD1990_to_USD2020 = 1.9802 # https://www.in2013dollars.com/us/inflation/1990?endYear=2020&amount=100
USD2005_to_USD2020 = 1.3252 # https://www.in2013dollars.com/us/inflation/2005?endYear=2020&amount=100
USD1990_to_USD2005 = 1.4943 # https://www.in2013dollars.com/us/inflation/2005?endYear=2020&amount=100

In [3]:
# Data for Figure 3 (Maps)
filepath = '/cluster/tufts/lamontagnelab/gkim14/P1_Sce_Analysis/Zenodo_Organization/'

basin_region_mapping = pd.read_csv(filepath + 'input_maps/basin_to_region_mapping.csv')
basin_mapping_key    = pd.read_csv(filepath + 'input_maps/basin_mapping_key.csv')

basin_mapping_key.Basin_long_name = basin_mapping_key.Basin_long_name.str.replace('_',' ',regex=True)
basin_mapping_key.Basin_long_name = basin_mapping_key.Basin_long_name.str.replace('-',' ',regex=True)
basin_mapping_key = basin_mapping_key.rename({'Basin_long_name':'basin'},axis=1)

GCAM_basin_region         = pd.read_csv(filepath + 'input_maps/GCAM_basin_region.csv')

basin_glu = GCAM_basin_region.Basin_GLU.to_numpy()
basin_glu_region = GCAM_basin_region.Region.to_numpy()

gcam_basins = gpd.read_file(filepath + 'input_maps/gcam_basins/gcam_basins.shp')
gcam_basins = gcam_basins.filter(['sbRgnAl','geometry','area','subRegn'])
gcam_basins = gcam_basins.rename(columns={'sbRgnAl':'Basin_ID','subRegn':'basin_full'})
gcam_basins['GLU_name'] = basin_mapping_key['GLU_name']

# 0) Good Runs

In [4]:
good_runs = pd.read_csv('Sep10_Data/good_runs_sep10_trial.csv')
good_runs = good_runs.sce.str.replace("kim_mcfe_","")
good_runs_list = good_runs.tolist()

In [5]:
incorrect_trial_runs = pd.read_csv('Sep10_Data/incorrect_trial_sep10_trial.csv', index_col = 0)
incorrect_trial_runs['scenario'] = incorrect_trial_runs.scenario.str.replace('[','')
incorrect_trial_runs['scenario'] = incorrect_trial_runs.scenario.str.replace(']','')

incorrect_trial_runs['scenario'] = incorrect_trial_runs.scenario.str.replace("'Starting new scenario: kim_mcfe_","")
incorrect_trial_runs['scenario'] = incorrect_trial_runs.scenario.str.replace("'","")
incorrect_trial_runs_list = incorrect_trial_runs.scenario.tolist()

In [6]:
good_runs_incorrect_trial_runs_list_merge = good_runs_list + incorrect_trial_runs_list

# 1) Calculate PWS & Limit to Good Runs

In [7]:
filepath = '/cluster/tufts/lamontagnelab/gkim14/MSD_Scenarios/Jupyter_Notebooks/Joo_database_MCFE_ensemble/Result_Analysis/Sep10_Data/'

ww = pd.read_csv(filepath+'ww_2050_ensemble.csv', index_col=[0])
ws = pd.read_csv(filepath+'ws_2050_ensemble.csv', index_col=[0])

In [8]:
ww = ww.assign(basin = lambda x: x.resource.str.replace('_water withdrawals',''))
ww_organized = ww.groupby(['scenario','basin','year'])['value'].sum().to_frame().reset_index()

ws_runoff = ws.assign(basin = lambda x: x.basin.str.replace('_water withdrawals',''))
ws_runoff_organized = ws_runoff.groupby(['scenario','basin','year'])['value'].sum().to_frame().reset_index()

pws = pd.merge(ww_organized,ws_runoff_organized, how="left", on=["scenario","basin","year"])
pws = pws.assign(physical_ws = lambda x: x.value_x / x.value_y)

pws.columns = ['scenario','basin','year','ww_all','ws_runoff','physical_ws']
pws.scenario = pws.scenario.str.replace('kim_mcfe_','',regex=True)

#pws = pws[pws.scenario.isin(good_runs_incorrect_trial_runs_list_merge)]
#pws.to_csv(filepath+'processed_data/2_physical_water_scarcity_basin_ensemble.csv')

In [9]:
pws = pws[pws.scenario.isin(good_runs_incorrect_trial_runs_list_merge)]

# 2) Get ag_prod, ag_price, and merge to ag_revenue by region

In [10]:
ag_prod = pd.read_csv(filepath+'ag_production_by_subsector_2050_ensemble.csv', index_col=[0])
ag_comm_prices = pd.read_csv(filepath+'ag_price_2050_ensemble.csv', index_col=[0])

In [11]:
ag_prod = ag_prod.drop_duplicates()
ag_comm_prices = ag_comm_prices.drop_duplicates()

In [12]:
ag_prod = ag_prod.groupby(['Units','scenario','region','sector','subsector','output','year'])['value'].sum().to_frame().reset_index()
ag_prod['Basin_GLU'] = ag_prod.subsector.str.split('_', expand=True)[1]
ag_prod = ag_prod.drop(['subsector','output'], axis=1)

In [13]:
# 20 Crops to calculate crop revenue (only exclude Forest)
crops_for_agprod = ['Corn', 'FiberCrop', 'FodderGrass', 'FodderHerb',
       'Fruits', 'Legumes', 'MiscCrop', 'NutsSeeds', 'OilCrop', 'OilPalm',
       'OtherGrain', 'Pasture', 'Rice', 'RootTuber', 'Soybean',
       'SugarCrop', 'Vegetables', 'Wheat', 'biomass']

crops_exclude_biomass = ['Corn', 'FiberCrop', 'FodderGrass', 'FodderHerb',
       'Fruits', 'Legumes', 'MiscCrop', 'NutsSeeds', 'OilCrop', 'OilPalm',
       'OtherGrain', 'Pasture', 'Rice', 'RootTuber', 'Soybean',
       'SugarCrop', 'Vegetables', 'Wheat']
biomass = ['biomass']

scenario_name = ag_comm_prices.scenario.unique()

In [14]:
############################### CREATE REGIONAL CROP TABLE #################################
# DIMENSION = 30,197,403 (3867 scenarios * 1 years * 411 basin-regions * 19 crops into account) #

regional_crop_prd = pd.DataFrame(index=range(3867*1*411*19), columns=range(5))
regional_crop_prd.columns = ['scenario','year','Basin_GLU','region','sector']

regional_crop_prd.scenario  = np.repeat(scenario_name,repeats=1*19*411,axis=0)
regional_crop_prd.year      = np.concatenate([np.repeat(2050,repeats=411*19,axis=0)]*3867)
regional_crop_prd.Basin_GLU = np.concatenate([np.repeat(GCAM_basin_region.Basin_GLU,repeats=19)]*3867*1)
regional_crop_prd.region    = np.concatenate([np.repeat(GCAM_basin_region.Region,repeats=19)]*3867*1)
regional_crop_prd.sector    = np.concatenate([crops_for_agprod]*(3867*1*411))

In [15]:
basin_region_crop_prd = pd.merge(regional_crop_prd, ag_prod, how='left',
                                 left_on=['scenario','year','Basin_GLU','region','sector'], 
                                 right_on=['scenario','year','Basin_GLU','region','sector'])

basin_region_crop_prd['value'] = basin_region_crop_prd['value'].fillna(0)

basin_region_crop_prd = pd.merge(basin_region_crop_prd, ag_comm_prices, how='left', 
                                 left_on=['scenario','year','region','sector'], 
                                 right_on=['scenario','year','region','sector'])
basin_region_crop_prd = basin_region_crop_prd.drop(['Units_x','Units_y'], axis=1)

In [16]:
basin_region_crop_prd = basin_region_crop_prd.drop_duplicates()

In [17]:
basin_region_crop_rev_crops   = basin_region_crop_prd[basin_region_crop_prd.sector.isin(crops_exclude_biomass)]
basin_region_crop_rev_biomass = basin_region_crop_prd[basin_region_crop_prd.sector.isin(biomass)]

basin_region_crop_rev_crops   = basin_region_crop_rev_crops.assign(Basin_Rev = lambda x: x.value_x * x.value_y * Mt_to_kg * USD1975_to_USD2020)
basin_region_crop_rev_biomass = basin_region_crop_rev_biomass.assign(Basin_Rev = lambda x: x.value_x * x.value_y * EJ_to_GJ * USD1975_to_USD2020)

basin_region_crop_rev_all = pd.concat([basin_region_crop_rev_crops,basin_region_crop_rev_biomass])

In [18]:
regional_crop_rev_crops = basin_region_crop_rev_all.groupby(['scenario','region','year'])['Basin_Rev'].sum().to_frame().reset_index()
basin_crop_rev_crops = basin_region_crop_rev_all.groupby(['scenario','Basin_GLU','region','year'])['Basin_Rev'].sum().to_frame().reset_index()


## Compute Ag Revenue Weighted PWS

In [19]:
basin_region_crop_rev = pd.merge(basin_crop_rev_crops, regional_crop_rev_crops, how='left', on=['scenario','region','year'])
basin_region_crop_rev.columns = ['scenario','Basin_GLU','region','year','Basin_Rev_all','Region_Rev_all']

basin_region_crop_rev = basin_region_crop_rev.assign(Crop_Rev_Ratio = lambda x: x.Basin_Rev_all / x.Region_Rev_all )

In [25]:
basin_region_crop_rev.scenario = basin_region_crop_rev.scenario.str.replace('kim_mcfe_','',regex=True)

basin_region_crop_rev_pws = pd.merge(basin_region_crop_rev, pws, how='left', 
                                     left_on=['scenario','Basin_GLU','year'], right_on=['scenario','basin','year'])
#basin_region_crop_rev_pws = basin_region_crop_rev_pws.filter(['scenario','Basin_GLU','region','year','Crop_Rev_Ratio','physical_ws'])
basin_region_crop_rev_pws = basin_region_crop_rev_pws.assign(wgt_reg_pws = lambda x: x.Crop_Rev_Ratio * x.physical_ws)

#basin_region_crop_rev_pws.to_csv('Sep10_Data/basin_region_pws_2050_full_ensemble.csv')

In [21]:
region_crop_rev_wgt_pws = basin_region_crop_rev_pws.groupby(['scenario','region','year'])['wgt_reg_pws'].sum().reset_index()
#region_crop_rev_wgt_pws.to_csv('Sep10_Data/region_pws_2050_full_ensemble.csv')

In [30]:
basin_region_crop_rev_pws[basin_region_crop_rev_pws['region']=="Pakistan"].head(20)

,scenario,Basin_GLU,region,year,Basin_Rev_all,Region_Rev_all,Crop_Rev_Ratio,basin,ww_all,ws_runoff,physical_ws,wgt_reg_pws
29,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,AmuDaryaR,Pakistan,2050,5.697376e+05,9.953636e+10,0.000006,AmuDaryaR,118.663000,98.9427,1.199310,0.000007
41,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,ArabianSea,Pakistan,2050,3.527376e+09,9.953636e+10,0.035438,ArabianSea,21.012100,28.6599,0.733153,0.025982
172,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,HamuMashR,Pakistan,2050,1.431564e+09,9.953636e+10,0.014382,HamuMashR,12.282500,21.0892,0.582407,0.008376
176,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,Helmand,Pakistan,2050,2.007704e+08,9.953636e+10,0.002017,Helmand,19.562500,36.3085,0.538786,0.001087
193,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,IndusR,Pakistan,2050,6.138819e+10,9.953636e+10,0.616741,IndusR,566.293670,249.1130,2.273240,1.402001
334,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,SabarmatiR,Pakistan,2050,3.297902e+10,9.953636e+10,0.331326,SabarmatiR,697.820000,219.7430,3.175619,1.052166
373,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,Tarim,Pakistan,2050,8.873412e+06,9.953636e+10,0.000089,Tarim,38.636400,251.1790,0.153820,0.000014
440,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd3,AmuDaryaR,Pakistan,2050,5.648386e+05,9.864373e+10,0.000006,AmuDaryaR,118.568000,98.9427,1.198350,0.000007
452,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd3,ArabianSea,Pakistan,2050,3.497354e+09,9.864373e+10,0.035454,ArabianSea,21.008700,28.6599,0.733035,0.025989
583,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd3,HamuMashR,Pakistan,2050,1.412379e+09,9.864373e+10,0.014318,HamuMashR,12.281900,21.0892,0.582379,0.008338


In [31]:
basin_region_crop_rev_pws[basin_region_crop_rev_pws['region']=="Central Asia"].head(20)

,scenario,Basin_GLU,region,year,Basin_Rev_all,Region_Rev_all,Crop_Rev_Ratio,basin,ww_all,ws_runoff,physical_ws,wgt_reg_pws
27,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,AmuDaryaR,Central Asia,2050,2.765321e+10,9.800689e+10,0.282156,AmuDaryaR,118.663000,98.9427,1.199310,0.338392
31,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,AmurR,Central Asia,2050,7.541314e+08,9.800689e+10,0.007695,AmurR,63.702100,557.6310,0.114237,0.000879
65,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,BlackSeaN,Central Asia,2050,2.146445e+09,9.800689e+10,0.021901,BlackSeaN,9.846390,83.5481,0.117853,0.002581
68,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,BlackSeaS,Central Asia,2050,1.057147e+09,9.800689e+10,0.010786,BlackSeaS,23.995833,108.6310,0.220893,0.002383
87,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,CaspianE,Central Asia,2050,6.425310e+09,9.800689e+10,0.065560,CaspianE,52.203438,30.8833,1.690345,0.110819
90,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,CaspianNE,Central Asia,2050,9.712654e+08,9.800689e+10,0.009910,CaspianNE,7.008780,47.4912,0.147581,0.001463
92,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,CaspianSW,Central Asia,2050,1.335732e+10,9.800689e+10,0.136290,CaspianSW,39.112438,55.8206,0.700681,0.095496
156,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,Gobi,Central Asia,2050,1.515799e+09,9.800689e+10,0.015466,Gobi,43.486000,93.8336,0.463437,0.007168
214,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,LBalkash,Central Asia,2050,5.648492e+09,9.800689e+10,0.057634,LBalkash,17.385000,34.4221,0.505053,0.029108
282,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,ObR,Central Asia,2050,5.963813e+09,9.800689e+10,0.060851,ObR,21.408400,601.3350,0.035601,0.002166


In [32]:
basin_region_crop_rev_pws[basin_region_crop_rev_pws['region']=="Middle East"].head(20)

,scenario,Basin_GLU,region,year,Basin_Rev_all,Region_Rev_all,Crop_Rev_Ratio,basin,ww_all,ws_runoff,physical_ws,wgt_reg_pws
8,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,AfrCstNE,Middle East,2050,3.728128e+08,2.761856e+11,0.001350,AfrCstNE,3.634284,38.974700,0.093247,0.000126
39,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,ArabianP,Middle East,2050,4.305037e+10,2.761856e+11,0.155875,ArabianP,71.489837,87.533000,0.816719,0.127306
40,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,ArabianSea,Middle East,2050,1.418876e+09,2.761856e+11,0.005137,ArabianSea,21.012100,28.659900,0.733153,0.003767
88,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,CaspianE,Middle East,2050,1.591221e+10,2.761856e+11,0.057614,CaspianE,52.203438,30.883300,1.690345,0.097388
94,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,CaspianSW,Middle East,2050,2.371145e+10,2.761856e+11,0.085853,CaspianSW,39.112438,55.820600,0.700681,0.060156
123,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,DeadSea,Middle East,2050,6.712586e+09,2.761856e+11,0.024305,DeadSea,5.643093,3.483180,1.620098,0.039376
135,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,EJrdnSyr,Middle East,2050,8.834175e+09,2.761856e+11,0.031986,EJrdnSyr,9.751752,5.051000,1.930658,0.061755
141,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,FarahrudR,Middle East,2050,1.467221e+09,2.761856e+11,0.005312,FarahrudR,4.767080,8.922460,0.534279,0.002838
171,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,HamuMashR,Middle East,2050,7.497075e+08,2.761856e+11,0.002715,HamuMashR,12.282500,21.089200,0.582407,0.001581
175,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,Helmand,Middle East,2050,1.157290e+08,2.761856e+11,0.000419,Helmand,19.562500,36.308500,0.538786,0.000226


,scenario,basin,year,ww_all,ws_runoff,physical_ws
0,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,AdnNicIsl,2050,0.000000,10.2516,0.000000
1,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,AdrBlkSea,2050,27.551200,254.6410,0.108196
2,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,AfrCstE,2050,8.870390,373.9120,0.023723
3,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,AfrCstNE,2050,3.634284,38.9747,0.093247
4,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,AfrCstNW,2050,40.382800,83.7870,0.481970
...,...,...,...,...,...,...
897139,ref_soc5_ag3_fd1_bldSatLevel_ipsl_res1_icd5,Yangtze,2050,210.210000,1682.7500,0.124921
897140,ref_soc5_ag3_fd1_bldSatLevel_ipsl_res1_icd5,YeniseiR,2050,7.704060,824.2740,0.009346
897141,ref_soc5_ag3_fd1_bldSatLevel_ipsl_res1_icd5,YucatanP,2050,6.324420,154.7620,0.040865
897142,ref_soc5_ag3_fd1_bldSatLevel_ipsl_res1_icd5,ZambeziR,2050,15.107200,330.2950,0.045739


In [44]:
idx_max_pws = pws.groupby(['scenario'])['physical_ws'].idxmax().to_frame().physical_ws
max_pws_bsn = pws.loc[pws.index.intersection(idx_max_pws),:]  
max_pws_bsn

,scenario,basin,year,ww_all,ws_runoff,physical_ws
192,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,SinaiP,2050,1.041597,0.209819,4.964264
424,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd3,SinaiP,2050,1.041283,0.209819,4.962767
656,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd5,SinaiP,2050,1.041825,0.209819,4.965352
888,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res1_icd2,SinaiP,2050,1.236820,0.325874,3.795393
1120,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res1_icd3,SinaiP,2050,1.236730,0.325874,3.795117
...,...,...,...,...,...,...
896163,ref_soc5_ag3_fd1_bldSatLevel_ipsl_res0_icd3,SabarmatiR,2050,438.005000,106.157000,4.126011
896395,ref_soc5_ag3_fd1_bldSatLevel_ipsl_res0_icd5,SabarmatiR,2050,447.920000,106.157000,4.219411
896640,ref_soc5_ag3_fd1_bldSatLevel_ipsl_res1_icd2,SinaiP,2050,1.410310,0.390832,3.608481
896872,ref_soc5_ag3_fd1_bldSatLevel_ipsl_res1_icd3,SinaiP,2050,1.411010,0.390832,3.610272


In [63]:
grouped = pws.groupby('scenario')['physical_ws']
pws['second_largest'] = grouped.transform(lambda x: x.nlargest(2).min())

In [64]:
pws[(pws['second_largest'])==(pws['physical_ws'])]

,scenario,basin,year,ww_all,ws_runoff,physical_ws,second_largest
179,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd2,SabarmatiR,2050,697.820000,219.743000,3.175619,3.175619
411,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd3,SabarmatiR,2050,680.827000,219.743000,3.098288,3.098288
643,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res0_icd5,SabarmatiR,2050,733.386000,219.743000,3.337472,3.337472
795,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res1_icd2,IndusR,2050,665.976474,298.553000,2.230681,2.230681
1027,ndc_soc2_ag1_fd0_bldBEESatLevel_canesm5_res1_icd3,IndusR,2050,665.980397,298.553000,2.230694,2.230694
...,...,...,...,...,...,...,...
896176,ref_soc5_ag3_fd1_bldSatLevel_ipsl_res0_icd3,SinaiP,2050,1.243024,0.324545,3.830052,3.830052
896408,ref_soc5_ag3_fd1_bldSatLevel_ipsl_res0_icd5,SinaiP,2050,1.241965,0.324545,3.826788,3.826788
896547,ref_soc5_ag3_fd1_bldSatLevel_ipsl_res1_icd2,IndusR,2050,665.182000,301.994000,2.202633,2.202633
896779,ref_soc5_ag3_fd1_bldSatLevel_ipsl_res1_icd3,IndusR,2050,663.864000,301.994000,2.198269,2.198269


In [60]:
grouped